In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols


In [3]:
# # Загрузка данных из текстового файла
# data = pd.read_csv('hypertension.txt', sep='\t')

# # Сохранение данных в Excel
# data.to_excel('hypertension.xlsx', index=False, engine='openpyxl')

# print("Файл успешно конвертирован в hypertension.xlsx!")

In [4]:
# Чтение данных из файла
data = pd.read_csv('hypertension.txt', sep='\t')


In [5]:
# Преобразование категориальных данных
data['psycho'] = data['psycho'].astype('category')
data['diet'] = data['diet'].astype('category')
data['drug'] = data['drug'].astype('category')

# Трёхфакторный дисперсионный анализ
model = ols('pressure ~ C(psycho) * C(diet) * C(drug)', data=data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

# Вывод результатов
print("Результаты трёхфакторного дисперсионного анализа:")
print(anova_table)

Результаты трёхфакторного дисперсионного анализа:
                           sum_sq    df          F        PR(>F)
C(psycho)                  2048.0   1.0  13.072340  6.150719e-04
C(diet)                    5202.0   1.0  33.204255  3.053308e-07
C(drug)                    3675.0   2.0  11.728723  5.018624e-05
C(psycho):C(diet)            32.0   1.0   0.204255  6.529374e-01
C(psycho):C(drug)           259.0   2.0   0.826596  4.424565e-01
C(diet):C(drug)             903.0   2.0   2.881915  6.381527e-02
C(psycho):C(diet):C(drug)  1075.0   2.0   3.430851  3.883423e-02
Residual                   9400.0  60.0        NaN           NaN


In [6]:
# Фильтрация данных для случаев, где применялась психотерапия
filtered_data = data[data['psycho'] == 'present']

# Двухфакторный дисперсионный анализ с учетом влияния diet и drug
model = ols('pressure ~ C(diet) + C(drug) + C(diet):C(drug)', data=filtered_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

# Вывод таблицы ANOVA
print("Таблица ANOVA:")
print(anova_table)

# Подробный отчет по модели
print("\nДетали модели:")
print(model.summary())




Таблица ANOVA:
                 sum_sq    df          F    PR(>F)
C(diet)          2209.0   1.0  15.248504  0.000496
C(drug)          2342.0   2.0   8.083295  0.001556
C(diet):C(drug)  1634.0   2.0   5.639669  0.008334
Residual         4346.0  30.0        NaN       NaN

Детали модели:
                            OLS Regression Results                            
Dep. Variable:               pressure   R-squared:                       0.587
Model:                            OLS   Adj. R-squared:                  0.519
Method:                 Least Squares   F-statistic:                     8.539
Date:                Fri, 03 Jan 2025   Prob (F-statistic):           4.06e-05
Time:                        04:49:16   Log-Likelihood:                -137.36
No. Observations:                  36   AIC:                             286.7
Df Residuals:                      30   BIC:                             296.2
Df Model:                           5                                         
Cov

In [7]:
# Шаг 3: Для пациентов, проходящих психотерапию
data_subset = data[data['psycho'] == 'present']

# Двусторонний ANOVA
model = ols('pressure ~ C(diet) * C(drug)', data=data_subset).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("\nРезультаты двухстороннего ANOVA:")
print(anova_table)

# Проверка значимости взаимодействий
if anova_table['PR(>F)']['C(diet):C(drug)'] < 0.05:
    print("\nЗначимое взаимодействие между диетой и препаратом. Выполняется односторонний ANOVA:")
    for diet_level in data_subset['diet'].unique():
        subset = data_subset[data_subset['diet'] == diet_level]
        model_single = ols('pressure ~ C(drug)', data=subset).fit()
        anova_single = sm.stats.anova_lm(model_single, typ=2)
        print(f"\nОдносторонний ANOVA для диеты {diet_level}:")
        print(anova_single)
else:
    print("\nВзаимодействие между диетой и препаратом незначимо. Дополнительный анализ не требуется.")
    


Результаты двухстороннего ANOVA:
                 sum_sq    df          F    PR(>F)
C(diet)          2209.0   1.0  15.248504  0.000496
C(drug)          2342.0   2.0   8.083295  0.001556
C(diet):C(drug)  1634.0   2.0   5.639669  0.008334
Residual         4346.0  30.0        NaN       NaN

Значимое взаимодействие между диетой и препаратом. Выполняется односторонний ANOVA:

Односторонний ANOVA для диеты absent:
          sum_sq    df          F    PR(>F)
C(drug)   3924.0   2.0  14.939086  0.000269
Residual  1970.0  15.0        NaN       NaN

Односторонний ANOVA для диеты present:
          sum_sq    df         F    PR(>F)
C(drug)     52.0   2.0  0.164141  0.850126
Residual  2376.0  15.0       NaN       NaN


In [8]:
# Определение наилучших условий лечения
means = data_subset.groupby(['diet', 'drug'])['pressure'].mean()
print("\nСредние значения давления для каждой комбинации диета-препарат:")
print(means)
best_condition = means.idxmin()
print(f"\nНаилучшие условия для снижения давления: Диета={best_condition[0]}, Препарат={best_condition[1]}")



Средние значения давления для каждой комбинации диета-препарат:
diet     drug
absent   1       168.0
         2       204.0
         3       189.0
present  1       169.0
         2       172.0
         3       173.0
Name: pressure, dtype: float64

Наилучшие условия для снижения давления: Диета=absent, Препарат=1


C:\Users\Varya02\AppData\Local\Temp\ipykernel_15772\230488531.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  means = data_subset.groupby(['diet', 'drug'])['pressure'].mean()


In [10]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Пример: данные по артериальному давлению, сгруппированные по факторам
data_subset = data[data['diet'] == 'absent']  # Подгруппа без диеты
tukey_results = pairwise_tukeyhsd(endog=data_subset['pressure'], 
                                  groups=data_subset['drug'], 
                                  alpha=0.05)

print(tukey_results)


 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     1      2     24.0 0.0006   9.8911 38.1089   True
     1      3     21.0 0.0025   6.8911 35.1089   True
     2      3     -3.0 0.8613 -17.1089 11.1089  False
-----------------------------------------------------
